<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
عیب یابی</font>
</h1>

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معرفی مجموعه داده
</font>
</h2>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>

</font>
</p>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
در این قسمت کتاب‌خانه‌ها و ابزار مورد نیاز خود را 
<code>import</code>
 کنید 
 و
  فایل داده‌ها را که در پوشه‌ی 
  <code>Data</code>
ذخیره‌شده‌اند را بخوانید و وارد محیط کار خود کنید.
</font>
</p>

In [11]:
# TODO:
import pandas as pd
import numpy as np
temp_train = pd.read_csv("Train/Temperature_C_train.csv", parse_dates=["timestamp"])
press_train = pd.read_csv("Train/Pressure_kPa_train.csv", parse_dates=["timestamp"])
vibacc_train = pd.read_csv("Train/VibAccel_m_s2_train.csv", parse_dates=["timestamp"])
vibvel_train = pd.read_csv("Train/VibVelocity_mm_s_train.csv", parse_dates=["timestamp"])
temp_test = pd.read_csv("Test/Temperature_C_test.csv", parse_dates=["timestamp"])
press_test = pd.read_csv("Test/Pressure_kPa_test.csv", parse_dates=["timestamp"])
vibacc_test = pd.read_csv("Test/VibAccel_m_s2_test.csv", parse_dates=["timestamp"])
vibvel_test = pd.read_csv("Test/VibVelocity_mm_s_test.csv", parse_dates=["timestamp"])


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
پیش‌پردازش و مهندسی ویژگی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
        در این سوال شما می‌توانید از هر تکنیک پیش‌پردازش/مهندسی ویژگی دلخواهتان، استفاده کنید.
    <br>
    تکنیک‌هایی که استفاده می‌کنید به شکل مستقیم مورد ارزیابی توسط سامانه داوری قرار <b>نمی‌گیرند.</b> بلکه همه آن‌ها در دقت مدل شما تاثیر خواهند گذاشت؛ بنابراین هر چه پیش‌پردازش/مهندسی ویژگی بهتری انجام دهید تا دقت مدل بهبود پیدا کند، امتیاز بیشتری از این سوال کسب خواهید کرد.
    در این قسمت شما می‌توانید بخشی از داده‌ی موجود را برای اعتبارسنجی در نظر بگیرید.
</font>
</p>

In [12]:
# TODO:
train = temp_train.merge(press_train, on="timestamp", how="outer")
train = train.merge(vibacc_train, on="timestamp", how="outer")
train = train.merge(vibvel_train, on="timestamp", how="outer")
test = temp_test.merge(press_test, on="timestamp", how="outer")
test = test.merge(vibacc_test, on="timestamp", how="outer")
test = test.merge(vibvel_test, on="timestamp", how="outer")
train = train.sort_values("timestamp").reset_index(drop=True)
test = test.sort_values("timestamp").reset_index(drop=True)
train = train.interpolate(method="linear")
test = test.interpolate(method="linear")
y = train["faulted"].astype("category").cat.codes  # normal=0, faulted=1
X = train.drop(columns=["faulted", "timestamp"])
X_test = test.drop(columns=["timestamp"])
for col in ["Temperature_C", "Pressure_kPa", "VibAccel_m_s2", "VibVelocity_mm_s"]:
    X[col+"_roll_mean"] = X[col].rolling(window=5, min_periods=1).mean()
    X[col+"_roll_std"]  = X[col].rolling(window=5, min_periods=1).std().fillna(0)
    X_test[col+"_roll_mean"] = X_test[col].rolling(window=5, min_periods=1).mean()
    X_test[col+"_roll_std"]  = X_test[col].rolling(window=5, min_periods=1).std().fillna(0)
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print("Train shape:", X_train.shape, "Validation shape:", X_val.shape, "Test shape:", X_test.shape)


Train shape: (85680, 12) Validation shape: (21420, 12) Test shape: (45900, 12)


C:\Users\Mahdie\AppData\Local\Temp\ipykernel_11876\892162991.py:10: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  train = train.interpolate(method="linear")


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
آموزش مدل
</font>
</h2>



In [13]:
from xgboost import XGBClassifier
labels = train["faulted"].dropna().unique()
print("Unique labels before mapping:", labels)
label_map = {"normal": 0, "faulted": 1}
y = train["faulted"].map(label_map)
mask = y.notna()
X_clean = X.loc[mask]
y_clean = y.loc[mask]
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X_clean, y_clean, test_size=0.2, random_state=42, stratify=y_clean
)
clf = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric="logloss",
    random_state=42,
    n_jobs=-1,
    use_label_encoder=False
)
clf.fit(X_train, y_train)
y_val_pred = clf.predict(X_val)


Unique labels before mapping: ['faulted' 'normal']


c:\Users\Mahdie\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:18:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معیار ارزیابی
</font>
</h2>



In [14]:
from sklearn.metrics import f1_score

# TODO:
from sklearn.metrics import classification_report
y_val_pred = clf.predict(X_val)
print(classification_report(y_val, y_val_pred, target_names=["normal", "faulted"]))
macro_f1 = f1_score(y_val, y_val_pred, average="macro")
print("Macro-F1:", macro_f1)


              precision    recall  f1-score   support

      normal       0.96      0.97      0.97      1196
     faulted       0.96      0.95      0.96       946

    accuracy                           0.96      2142
   macro avg       0.96      0.96      0.96      2142
weighted avg       0.96      0.96      0.96      2142

Macro-F1: 0.9606488710892289


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
 پیش‌بینی برای داده تست و خروجی
</font>
</h2>


In [15]:
# TODO:

# submission = None
vibacc_test = pd.read_csv("Test/VibAccel_m_s2_test.csv", parse_dates=["timestamp"])
X_test_final = X_test.loc[vibacc_test.index]
y_test_pred = clf.predict(X_test_final)
inv_map = {0: "normal", 1: "faulted"}
submission = pd.DataFrame({"prediction": pd.Series(y_test_pred).map(inv_map)})
submission.to_csv("submission.csv", index=False)
print("Submission size:", submission.shape)  # باید 4590 باشه


Submission size: (4590, 1)


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
<b>سلول جواب‌ساز</b>
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    برای ساخته‌شدن فایل <code>result.zip</code> سلول زیر را اجرا کنید. توجه داشته باشید که پیش از اجرای سلول زیر تغییرات اعمال شده در نت‌بوک را ذخیره کرده باشید (<code>ctrl+s</code>) در غیر این صورت، در پایان مسابقه نمره شما به صفر تغییر خواهد کرد.
    <br>
    همچنین اگر از کولب برای اجرای این فایل نوت‌بوک استفاده می‌کنید، قبل از ارسال فایل <code>result.zip</code>، آخرین نسخه‌ی نوت‌بوک خود را دانلود کرده و داخل فایل ارسالی قرار دهید.
</font>

In [16]:
import zipfile
import joblib
import os

if not os.path.exists(os.path.join(os.getcwd(), 'notebook.ipynb')):
    %notebook -e notebook.ipynb

def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)

submission.to_csv('submission.csv', index=False)
file_names = ['notebook.ipynb', 'submission.csv']
compress(file_names)

File Paths:
['notebook.ipynb', 'submission.csv']
